In [147]:
# Блок 7
# Функции и их переменные

import os
path_data = "../data"

SOURCE_TYPE = "sourceType"
ORDER_TYPE = "orderType"
ID = 0
DATE = 1
USER_ID = 2
DURATION = 3
MEDIUM = 4
SOURCE = 5
COST = 6
ORDER_ID = 7
AMOUNT_PAID = 8
headers = ["id", "date", "user_id",	"duration",	"medium", "source", "cost", "order_id", "amount_paid"]

In [148]:
# создаём словарь статистики
def calcSourceType(total: dict, current: dict) -> dict:
    """
    Функция сбора статистики  по типу источника
    
    :param total: 
    :param current: 
    :return: 
    """
    _total = total.copy()
    value = current.get(SOURCE_TYPE)
    _total.update({value: _total.get(value, 0) + 1})
    return _total

In [149]:
def calcSourceOrderType(total: dict, current: dict) -> dict:
    """
    
    :param total: 
    :param current: 
    :return: 
    """
    _total = total.copy()
    source_type_value = current.get(SOURCE_TYPE)
    order_type_value = current.get(ORDER_TYPE)
    value = f"{source_type_value} {order_type_value}"
    _total.update({value: _total.get(value, 0) + 1})
    return _total

In [150]:
def source_type_classification(source, medium):
    """

    Функция классифицирует источник трафика в зависимости от значения source и medium.

    Возвращает название типа источника по правилам:

        - если source равен 'google' или 'yandex', то проверяем medium:

            - для medium 'seo' или 'sem' возвращаем 'search engines seo'

            - для medium 'brand' - возвращаем 'search engines brand'

            - для остальных случаев возвращаем 'search engines undefined'

        - если условие не выполнено, то возвращаем 'other'

    """
    if source == "google" or source == "yandex":
        if medium == 'seo' or medium == 'sem':
            sourceType = 'search engines seo'
        elif medium == 'brand':
            sourceType = 'search engines brand'
        else:
            sourceType = 'search engines undefined'
    else:
        sourceType = "other"
    return sourceType

In [151]:
def order_type_classification(amount_paid):
    if amount_paid >= 20:
        orderType = 'expensive' 
    else:
        orderType = 'cheap'
    return orderType

In [152]:
# фильтруем список
def sift(line) -> dict:
    """
    Функция возращает базовый элементы для сборка статистика.
    
    :param line: 
    :return: 
    """
    source, medium, amount = line[SOURCE], line[MEDIUM], float(line[-1].replace(',', '.'))
    order_type = order_type_classification(amount)
    source_type = source_type_classification(source, medium)
    return {ORDER_TYPE: order_type, SOURCE_TYPE: source_type}

In [153]:
# основная функция подчёт 
def proc_stats(filename, calcStats, condition) -> dict:
    """
    функция подчёта статистика
    
    :param filename: 
    :param condition: 
    :return: 
    """
    
    total_stats: dict = {}
    with open(os.path.abspath(filename), "r") as file:
        for line in file:
            line = line.strip().split('\t')
            total_stats = calcStats(total_stats, condition(line))
    return total_stats

In [154]:
proc_stats(f"{path_data}/data_no_header.txt", calcSourceType, sift)

{'other': 113, 'search engines brand': 13, 'search engines seo': 168}

In [155]:
source_type_classification("google", "123")

'search engines undefined'

In [156]:
proc_stats(f"{path_data}/data_no_header.txt", calcSourceOrderType, sift)

{'other cheap': 74,
 'other expensive': 39,
 'search engines brand cheap': 9,
 'search engines brand expensive': 4,
 'search engines seo cheap': 121,
 'search engines seo expensive': 47}